# __Pipeline 1:__
## _Normalization --> Z Score --> 3 STD_

## Importing Data:

In [ ]:
# import required package for data handling
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import matplotlib.pyplot as plt
from pandas import DataFrame
%matplotlib inline

# import required packages for splitting data
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# import required packages for evaluating models
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# import `logistic regression` model
from sklearn.linear_model import LogisticRegression

In [ ]:
compdata_original = pd.read_csv('comp1data.csv', header=0, na_values = '-')
compdata_original.head()

# Descriptive Statistics

In [ ]:
compdata_original.shape
# 682 rows and 22 columns

In [ ]:
compdata_original.describe()

In [ ]:
compdata_original.dtypes

In [ ]:
compdata_original.median()

In [ ]:
# Create a copy of the data
compdata = compdata_original.copy()

# Variable Calculations:

In [ ]:
# Impute the rows T2 values!!!! mean
# Cannot do calculations with missing data on the number of words, the ratios will be 0 and create outliers

compdata.loc[compdata['T2'] == 0]

In [ ]:
compdata.loc[125,'T2']=compdata['T2'].mean()

In [ ]:
compdata.loc[199,'T2']=compdata['T2'].mean()

In [ ]:
# check for negatives
# cannot have a negative number of long words (T5)

compdata.loc[compdata['T5'] < 0]

In [ ]:
compdata.loc[7,'T5']=compdata['T5'].mean()

In [ ]:
# cannot have a negative number of positive words (S1)
compdata.loc[compdata['S1'] < 0]

In [ ]:
# Fill the 0, negative, and infinite values with the mean for continuous values
compdata.loc[134,'S1']=compdata['S1'].mean()

In [ ]:
compdata['S1'] = pd.to_numeric(compdata['S1'])

In [ ]:
# Ratios:

    # ** How to find the ratio for T1 and T2... impute them

# Percentage of real words
compdata['T3_'] = (compdata['T3']/compdata['T2'])
# Percentage of long sentences
compdata['T4_'] = (compdata['T4']/compdata['T1'])
# Percentage of long words
compdata['T5_'] = (compdata['T5']/compdata['T2'])
# Percentage of positive words
compdata['S1_'] = (compdata['S1']/compdata['T2'])
# Percentage of negative words
compdata['S2_'] = (compdata['S2']/compdata['T2'])
# Percentage of uncertain words
compdata['S3_'] = (compdata['S3']/compdata['T2'])
compdata.head()

# Target & Control Variable Calculation

compdata['P(mid)'] = ((compdata['P(H)']+compdata['P(L)'])/2)

def p(row):
    if row['P(IPO)'] < row['P(mid)']:
        val = 1
    else:
        val = 0
    return val

def q(row):
    if row['P(1Day)'] > row['P(IPO)']:
        val = 1
    else:
        val = 0
    return val

def r(row):
    if row['C3'] >= 0:
        val = 1
    else:
        val = 0
    return val

def s(row):
    if row['P(IPO)'] > row['P(mid)']:
        val = (row['P(IPO)'] - row['P(mid)'])/row['P(mid)']*100
    else:
        val = 0
    return val

compdata['C3_'] = compdata.apply(r, axis=1)
compdata['C5_'] = compdata['C5']/compdata['C6'] 
compdata['C6_'] = compdata.apply(s, axis=1)
compdata['Y1'] = compdata.apply(p, axis=1)
compdata['Y2'] = compdata.apply(q, axis=1)

# Missing Data:

In [ ]:
compdata.info()

In [ ]:
compdata.isna().sum()

In [ ]:
# what is the percentage of missing data in each column
compdata.isna().mean().round(4) * 100

# Majority of the data is missing from C7

In [ ]:
#isinstance(compdata['I3'], object) 

#compdata['I3'] = [isinstance(x, str) for x in compdata.I3]
#compdata

#compdata[compdata.I3.apply(lambda x: isinstance(x, str))]

In [ ]:
compdata[compdata["I1"]=='DLB']

In [ ]:
compdata[compdata["I1"]=='DTSI']

In [ ]:
compdata[compdata["I1"]=='RLD']

In [ ]:
compdata[compdata["I1"]=='UPG']

In [ ]:
compdata.I3.replace(to_replace ="6794, 3861, 3663, 7819", value ="3000", inplace=True)

In [ ]:
compdata.I3.replace(to_replace ="3651, 6794, 7819", value ="9900", inplace=True)

In [ ]:
compdata.I3.replace(to_replace ="3663, 3861, 6794", value ="3000", inplace=True)

In [ ]:
compdata.I3.replace(to_replace ="7389, 5063", value ="9900", inplace=True)

In [ ]:
compdata.I3.replace(to_replace ="541990", value ="7000", inplace=True)

In [ ]:
compdata["I3"] = pd.to_numeric(compdata["I3"])

In [ ]:
def assign_I3_bins(I3):
    if I3 >= 100 and I3 <= 999:                    
        return(1)
    if I3 >= 1000 and I3 <= 1499:       
        return(2)
    if I3 >= 1500 and I3 <= 1799:       
        return(3)
    if I3 >= 1800 and I3 <= 1999:      
        return(4)
    if I3 >= 2000 and I3 <= 3999:                  
        return(5)
    if I3 >= 4000 and I3 <= 4999:                  
        return(6)
    if I3 >= 5000 and I3 <= 5199:                 
        return(7)
    if I3 >= 5200 and I3 <= 5999:                  
        return(8)
    if I3 >= 6000 and I3 <= 6799:                 
        return(9)
    if I3 >= 7000 and I3 <= 8999:                  
        return(10)
    if I3 >= 9100 and I3 <= 9729:                  
        return(11)   
    if I3 >= 9900 and I3 <= 9999:                  
        return(12)   
    
    # Creating a new column I3_bins in the dataframe

compdata['I3_bins'] = compdata['I3'].apply(assign_I3_bins)  

compdata.head()

In [ ]:
compdata.isna().sum()

In [ ]:
# Since C2 is a binary value, we will fill the missing data with the mode

compdata['Imputed_C2'] = compdata['C2'].fillna(compdata['C2'].mode()[0])
compdata.head()

In [ ]:
compdata['Imputed_C7'] = compdata['C7'].fillna(compdata['C7'].mean())

In [ ]:
compdata.isna().sum()

In [ ]:
# Drop missing P Values from Continues DF
compdata = compdata.dropna(subset=['P(1Day)'], axis=0, how='any')

In [ ]:
compdata.reset_index(drop=True, inplace=True)
compdata.head(20)

In [ ]:
# Create DF with only continuous features

# Drop the imputed variable colmuns
# Drop the P values
# Drop Y1 Y2 **make seperate df
    # Drop T1 and T2?

compdata_cont = compdata.drop(['P(IPO)','P(H)','P(L)','P(1Day)','P(mid)','I1','I2','I3','T3','T1','T2','T4','T5','C3_','S1','S2','S3','C2','C3','C5','C6','C7','Imputed_C2','I3_bins','Y1','Y2'], axis=1)
compdata_cont.head()

In [ ]:
compdata_cont.isna().sum()

In [ ]:
# Create DF with only categorical features
# drop y1 y2 c2 i3
compdata_cat = compdata.drop(['P(IPO)','P(H)','P(L)','P(1Day)','P(mid)','I3','C2','C1','C3','C4','C5','C6','C7','T1','T2','T3','T4','T5','S1','S2','S3','T3_','T4_','T5_','S1_','S2_','S3_','C5_','C6_','Y1','Y2','Imputed_C7'], axis=1)
compdata_cat.head(15)

In [ ]:
compdata_cat.shape

In [ ]:
targetvals = compdata.drop(['P(IPO)','P(H)','P(L)','P(1Day)','P(mid)','I3','C2','C1','C3','C4','C5','C6','C7','T1','T2','T3','T4','T5','S1','S2','S3','T3_','T4_','T5_','S1_','S2_','S3_','C5_','C6_','I1','I2','C3_','I3_bins','Imputed_C2'], axis=1)
targetvals.head()

#### Run evaluation code now.. go back and check the results to see if it improved or not

##### after impute, after min max, after normalization

# Pipeline 1: Normalization -> Z Score -> 3 STD

## Normalization

## Skewness 

In [ ]:
compdata_cont.max()

In [ ]:
# library and data
import seaborn as sns
import matplotlib.pyplot as plt
 
# plot
f, axes = plt.subplots(3, 4, figsize=(15,10), sharex=False)
sns.distplot( compdata_cont["C1"] , ax=axes[0, 0])
sns.distplot( compdata_cont["C4"] , ax=axes[0, 1])
sns.distplot( compdata_cont["T3_"] , ax=axes[0, 2])
sns.distplot( compdata_cont["T4_"] , ax=axes[0, 3])
sns.distplot( compdata_cont["T5_"] , ax=axes[1, 0])
sns.distplot( compdata_cont["S1_"] , ax=axes[1, 1])
sns.distplot( compdata_cont["S2_"] , ax=axes[1, 2])
sns.distplot( compdata_cont["S3_"] , ax=axes[1, 3])
sns.distplot( compdata_cont["C5_"] , ax=axes[2, 0])
sns.distplot( compdata_cont["C6_"] , ax=axes[2, 1])
sns.distplot( compdata_cont["Imputed_C7"] , ax=axes[2, 2])
plt.show()

In [ ]:
compdata_cont.skew()

# check if its left or right skewed

In [ ]:
# Positive skew

compdata_cont['C1'] = compdata_cont['C1'].apply(np.cbrt)
compdata_cont['C1'] = compdata_cont['C1'].apply(np.cbrt)
compdata_cont['C1'].skew()

In [ ]:
# Negative skew

compdata_cont['T3_'] = compdata_cont['T3_'].apply(np.exp)
compdata_cont['T3_'].skew()

In [ ]:
# Negative skew

compdata_cont['T4_'] = compdata_cont['T4_'].apply(np.exp)
compdata_cont['T4_'] = compdata_cont['T4_'].apply(np.exp)
compdata_cont['T4_'].skew()

In [ ]:
# Positive skew

compdata_cont['S1_'] = compdata_cont['S1_'].apply(np.log)
compdata_cont['S1_'].skew()

In [ ]:
# Positive skew

compdata_cont['S2_'] = compdata_cont['S2_'].apply(np.log)
compdata_cont['S2_'].skew()

In [ ]:
# Positive skew

compdata_cont['S3_'] = compdata_cont['S3_'].apply(np.log)
compdata_cont['S3_'].skew()

In [ ]:
# Positive skew

compdata_cont['C5_'] = compdata_cont['C5_'].apply(np.cbrt)
compdata_cont['C5_'] = compdata_cont['C5_'].apply(np.cbrt)
compdata_cont['C5_'].skew()

In [ ]:
# Positive skew

compdata_cont['C6_'] = compdata_cont['C6_'].apply(np.cbrt)
compdata_cont['C6_'].skew()

In [ ]:
# library and data
import seaborn as sns
import matplotlib.pyplot as plt
 
# plot
f, axes = plt.subplots(3, 4, figsize=(15,10), sharex=False)
sns.distplot( compdata_cont["C1"] , ax=axes[0, 0])
sns.distplot( compdata_cont["C4"] , ax=axes[0, 1])
sns.distplot( compdata_cont["T3_"] , ax=axes[0, 2])
sns.distplot( compdata_cont["T4_"] , ax=axes[0, 3])
sns.distplot( compdata_cont["T5_"] , ax=axes[1, 0])
sns.distplot( compdata_cont["S1_"] , ax=axes[1, 1])
sns.distplot( compdata_cont["S2_"] , ax=axes[1, 2])
sns.distplot( compdata_cont["S3_"] , ax=axes[1, 3])
sns.distplot( compdata_cont["C5_"] , ax=axes[2, 0])
sns.distplot( compdata_cont["C6_"] , ax=axes[2, 1])
sns.distplot( compdata_cont["Imputed_C7"] , ax=axes[2, 2])
plt.show()

In [ ]:
# check for infinite numbers in df

#close to 0 is normal (T4_).. [-.5 - .5]


# log all the values for the continues values *positives
# exponential for the negative continues values

# make it fit better into range after log and exponential (sqrt? log again?)

# get in range ***

# Z Score

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
print(scaler.fit(compdata_cont))

In [ ]:
print(scaler.mean_)

In [ ]:
print(scaler.transform(compdata_cont))

In [ ]:
from scipy.stats import zscore

compdata_cont = compdata_cont.apply(zscore)
compdata_cont.head()

# 3 STD

In [ ]:
for col in compdata_cont.columns:
    u_bound = compdata_cont[col].mean() + 3* compdata_cont[col].std()
    compdata_cont[col][compdata_cont[col] > u_bound] = u_bound

In [ ]:
for col in compdata_cont.columns:
    u_bound = compdata_cont[col].mean() - 3* compdata_cont[col].std()
    compdata_cont[col][compdata_cont[col] < l_bound] = l_bound

# Correlation

In [ ]:
# Correlation between Variables

index= ['P(IPO)', 'P(H)', 'P(L)', 'P(1Day)','C1','C3','C4','C5','C6','C7','T1','T2','T3','T4','T5','S1','S2','S3']
cols = ['P(IPO)', 'P(H)', 'P(L)', 'P(1Day)','C1','C3','C4','C5','C6','C7','T1','T2','T3','T4','T5','S1','S2','S3']
df = DataFrame(abs(np.random.randn(18,18)), index=index,  columns=cols)
fig, ax = plt.subplots(figsize=(15,15))
sns.heatmap(df, annot=True, cbar=True, cmap='Blues')
plt.title("Correlation between Variables")
plt.show()

# look for the ones over .5 or under -.5.. these are highly correlated ones
# flagging step

#every pair highly correlated need to be seperated into a different df

# make differe
    # 1 pair = 2
    # 2 pair = 4

In [ ]:
# explain why theyre seperated

## do feature selection

# correlation is alwasy between -1 and 1

# merge all 3 parts together categorical back into .. categorical, contin (all the continues made.. not 1), target

# zscore

>>> from sklearn.preprocessing import StandardScaler
>>> data = [[0, 0], [0, 0], [1, 1], [1, 1]]
>>> scaler = StandardScaler()
>>> print(scaler.fit(data))
StandardScaler()
>>> print(scaler.mean_)
[0.5 0.5]
>>> print(scaler.transform(data))
[[-1. -1.]
 [-1. -1.]
 [ 1.  1.]
 [ 1.  1.]]
>>> print(scaler.transform([[2, 2]]))
[[3. 3.]]

- report all we did to change the data, not the data describing that we used to impute and change
- keep count of missing